# IDENTIFICACIÓN DE SISTEMAS

Profesor: Jairo Alberto Cuéllar Guarnizo  
Programa: Ingeniería en Automatización y Control

In [5]:
from sympy import MatrixSymbol, Matrix, Identity

import sympy as sym
import pandas as pd

sym.init_printing()
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import control
import random

# Métodos paramétricos de estimación

## Estimación Lineal por mínimos cuadráticos (LS) - Resumen

Para el siguiente modelo lineal:

$$y[k] = \phi[k]^T\theta + \varepsilon[k]$$

Inicialmente consideremos el error como 0, por tanto el modelo se reduce a $y[k] = \phi[k]^T\theta$. La idea de LLS es minimizar el valor de error entre las mediciones $y[k]$ y los valores estimados $\theta$ elevados al cuadrado, por tanto la función de costo a minimizar sería:

$$f(\theta_{LS}) = \frac{1}{2}|y - \phi\theta|^2$$

En la sección anterior se llegó a la conclusión que el vector de parámetros estimados es equivalente a:

$$\theta = [\phi^T.\phi]^{-1}.\phi^T.y$$

$$\theta_{LS} = \phi^+.y$$

## Estimación Lineal por mínimos cuadráticos con pesos (WLS)

Considerando que la señal del error es de media cero, pero que contiene varianzas distintas, se podría evaluar la siguiente función de costos, que considera para cada medición se divida en la varianza de dicha medición. Por tanto la función de costo no tendría unidades:

$$f_{WLS}(\theta) = \sum_{k=1}^{N}\frac{(y[k]-\phi^T.\theta)^2}{\sigma_{\epsilon}[k]^2}$$

En este caso, se puede introducir una matriz de pesos, denominada W y que estaría dada por la siguiente relación:

$$W = \begin{bmatrix}\sigma_{\epsilon}[1]^{-2}& & \\ & ... & \\ & &\sigma_{\epsilon}[N]^{-2}\end{bmatrix}$$

Se podría reescribir la función de costo de forma vectorial así:

$$f_{WLS}(\theta) = |y-\phi.\theta|^2.W$$

$$f_{WLS}(\theta) = (y-\phi.\theta)^T.W.(y-\phi.\theta)$$

Calculando el gradiente en función de teta y despejando el estimador se obtiene la siguiente relación:

$$\theta_{WLS} = (\phi^T.W.\phi)^{-1}.\phi^T.W.y$$

Otra forma de obtener el vector de los parámetros estimados es introducir la matriz $W^{1/2}$, que es equivalente a la raiz cuadrada de la matriz de pesos W.

$$W^{1/2} = \begin{bmatrix}\sigma_{\epsilon}[1]^{-1}& & \\ & ... & \\ & &\sigma_{\epsilon}[N]^{-1}\end{bmatrix}$$

Tal que: $W = W^{1/2}.W^{1/2}$

Sustituyendo en la función de costo se tiene lo siguiente:

$$f_{WLS}(\theta) = |W^{1/2}.(y-\phi.\theta)|^2 = |W^{1/2}.y-W^{1/2}.\phi.\theta)|^2 $$

Se introducen dos nuevos vectores como se indica a continuación para el vector normalizados de regresión y para los valores medidos normalizados:

$$\tilde{y} = W^{1/2}.y$$

$$\tilde{\phi} = W^{1/2}.\phi$$

Aplicando los criterios de optimalidad a esta función de costo (Calcular gradiente en función de $\theta$, despejando el estimador y teniendo en cuenta las 2 relaciones anteriores se podría indicar que el vector de parámetros estimados daría:

$$\theta_{WLS} = [\tilde{\phi}^T.\tilde{\phi}]^{-1}.\tilde{\phi}^T.\tilde{y}$$

$$\theta_{WLS} = \tilde{\phi}^+.\tilde{y}$$

Así que un problema de mínimos cuadrados con pesos, es solo un problema de mínimos cuadrados escalando la matriz de regresión y los valores medidos por la matriz $W^{1/2}$.

## Ejemplo 1:

Se espera poder determinar los parámetros de un modelo lineal dado por: $y = -6 + 2x$. Además, se tiene un vector de mediciones de longitud 5, dado por la siguiente relación:
$$y[k] = \begin{bmatrix}y[1]\\y[2]\\...\\y[N]\end{bmatrix}=\begin{bmatrix}-5.996\\-4.008\\-1.997\\-0.009 \\ 2.009\end{bmatrix}$$.

Este vector ha sido contaminado por ruido y contiene error. Considere a su vez que x va de 0 a 4.

In [6]:
#Valores de x = 0, 1, 2, 3, 4
N = 5
x = np.linspace(0,4,5,dtype = "int")

#Valores medidos con un error y
y_v = np.array([-6,-4,-2,0,2])
y = np.array([0.007,35.096,64.409,75.9,81.609])
x, y

(array([0, 1, 2, 3, 4]),
 array([7.0000e-03, 3.5096e+01, 6.4409e+01, 7.5900e+01, 8.1609e+01]))

La matriz de regresión, al ser un modelo lineal estaría dada por: $\phi = \begin{bmatrix}1 & x[1]\\1 & x[2]\\1 & x[3]\\1 & x[4]\\1 & x[5]\end{bmatrix}$. 

Y se debe considerar las covarianzas dadas por:

$W = \begin{bmatrix}0.063&0&0&0&0\\0&0.062&0&0&0\\0&0&0.25&0&0\\0&0&0&12345&0\\0&0&0&0&0.248\end{bmatrix}$

In [10]:
# La matriz de regresión considerando el modelo lineal
ones = np.ones(N)
phi = np.array([ones,x,x**2]).transpose()

# Varianzas para cada xk
#var = np.array([0.063,0.062,0.251,15625,0.25])
var = np.array([10,10,10,10,10])
_var = var**0.5

## Matriz de pesos (Covarianzas)
W = np.eye(N)*var
_W = np.eye(N)*_var
W,    _W

(array([[10.,  0.,  0.,  0.,  0.],
        [ 0., 10.,  0.,  0.,  0.],
        [ 0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0., 10.,  0.],
        [ 0.,  0.,  0.,  0., 10.]]),
 array([[3.16227766, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 3.16227766, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 3.16227766, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 3.16227766, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 3.16227766]]))

Se debe calcular los vectores normalizados 
$$\tilde{y} = W^{1/2}.y$$

$$\tilde{\phi} = W^{1/2}.\phi$$

In [11]:
## Vectores "y" y "phi" Normalizados usando W
_y = np.dot(_W,y)
_phi = np.dot(_W,phi)
_y,      _phi

(array([2.21359436e-02, 1.10983297e+02, 2.03679142e+02, 2.40016874e+02,
        2.58070318e+02]),
 array([[ 3.16227766,  0.        ,  0.        ],
        [ 3.16227766,  3.16227766,  3.16227766],
        [ 3.16227766,  6.32455532, 12.64911064],
        [ 3.16227766,  9.48683298, 28.46049894],
        [ 3.16227766, 12.64911064, 50.59644256]]))

In [12]:
## Calculando los estimados THETA
phi_pr = np.dot(_phi.transpose(),_phi)
A = np.linalg.inv(phi_pr+0.01*np.eye(N))
B = np.dot(_phi.transpose(),_y)

## Vector de parámetros estimados
theta = np.dot(A,B)
theta

array([-0.33768571, 42.28137143, -5.47014286])

In [5]:
##  Estimación vs Medición
y_e = theta[0] + theta[1]*x

plt.figure(figsize=(8,4))
plt.plot(x,y_v)
plt.scatter(x,y)
plt.plot(x,y_e)

plt.legend(["Ideal","Medición","Estimado"])
plt.grid()

NameError: name 'theta' is not defined

## Regularización de mínimos cuadráticos:

Un problema mal planteado de mínimos cuadrados puede originarse en el momento que la matriz MorePenrose no es invertible, o que la inversa de $[\tilde{\phi}^T.\tilde{\phi}]$ no existe. La regularización del problema restringe un poco más la solución del vector de parámetros estimados y plantea una posible solución de norma mínima. Depende de la escogencia de valores apropiados, pero el problema a resolver sería el siguiente:

$$f(\theta_{LS}) = \frac{1}{2}|y - \phi\theta|^2+ \frac{\alpha}{2}|\theta|^2$$


Minimizando la función objetivo e igualando a cero el gradiente:

$$\bigtriangledown f[\theta_{LS}] = -\phi^T.y + \phi^T.\phi.\theta + \alpha.\theta = 0$$

Despejando el valor de los parámetros:

$$\theta_{LS} = (\phi^T.\phi+\alpha.I)^{-1}.\phi^T.y$$

Recordar que para este caso, el vector $\phi$ hace referencia al vector de estimación y a las mediciones afectadas por los pesos en la sección anterior. Si se hace $\alpha = 0$ la expresión queda idéntica a la de mínimos cuadrados con pesos.

## [Ejercicio en Clase 3](ejercicio_03.ipynb)